In [1]:
import json
import string
import requests
from requests.exceptions import Timeout
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from scipy.stats import linregress
import csv 
import io


In [2]:
######get datasets to explore 
#beginning year

#/FARSData/GetFARSData?dataset=Accident&FromYear=2014&ToYear=2016&State=1&format=csv

dataset = ['ACCIDENT','DISTRACT','PERSON','VIOLATION']
baseurl = 'https://crashviewer.nhtsa.dot.gov/CrashAPI/FARSData/GetFARSData'

for x in dataset:
    yearloop = 2010
    params = {'states':'1,51',
            'FromYear':yearloop,
            'ToYear':yearloop,
            'dataset':x,
            'format':'csv'}
    for i in range (12):
        #wait up to 70 seconds for responding data
        try:
            response = requests.get(baseurl,params=params,timeout=(10,70))
        except Timeout:
            print('The request timed out')

        if response.status_code !=200:
            print('There is error')
        else:
            #get response as CSV
            response.encoding = response.apparent_encoding
            #get data in string
            content = response.text
            print(yearloop)
            #convert string to pandas dataframe
            temp_data = pd.read_csv(io.StringIO(content))
            if i ==0:
                #response_json = response.iter_lines()
                #save data to fatality_data if 1st data
                fatality_data = pd.read_csv(io.StringIO(content))           
            else:
                #response_json = response.iter_lines()
                #append 2nd data point on ward to fatality_data
                temp_data = pd.read_csv(io.StringIO(content))
                fatality_data = pd.concat([fatality_data,temp_data],ignore_index=True) 
        #update parameter for next year
        yearloop += 1
        params = {'states':'1,51',
            'FromYear':yearloop,
            'ToYear':yearloop,
            'dataset':x,
            'format':'csv'}

    #export final data to csv
    fatality_data.to_csv(f'data/{x}.csv',index=False)

2010
2011
2012
2013


c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (40,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2014
2015


c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (40,42,71) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2016
2017
2018
2019
2020
2021
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [3]:
###### trim off data to reduce the file as the raw data is huge (>1GB)
df = pd.read_csv('data/ACCIDENT.csv')
df =df[['caseyear','st_case','statename','latitude','longitud','fatals','lgt_condname','drunk_dr','fatals','day_weekname','day_week']]
df.to_csv('data/ACCIDENT.csv',index=False)
df.head()


c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (28,30,32,58,64,66,86,88,90) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,caseyear,st_case,statename,latitude,longitud,fatals,lgt_condname,drunk_dr,fatals,day_weekname,day_week
0,2010,10001,Alabama,32.641064,-85.354692,1,Dark - Lighted,1,1,Friday,6
1,2010,10002,Alabama,31.430447,-86.956694,1,Dark - Lighted,0,1,Monday,2
2,2010,10003,Alabama,30.691631,-88.085778,1,Daylight,0,1,Thursday,5
3,2010,10004,Alabama,33.868700,-86.291164,1,Dark - Lighted,0,1,Thursday,5
4,2010,10005,Alabama,33.309742,-86.787222,1,Dawn,0,1,Monday,2


In [4]:
#trim off data
df=pd.read_csv('data/DISTRACT.csv')
df=df[['caseyear','st_case','statename','drdistract','drdistractname']]
#remove duplicate values
df=df.sort_values('drdistract',ascending=False)
df=df.drop_duplicates(subset=['caseyear','st_case'])
df.to_csv('data/DISTRACT.csv',index=False)
df.head()


,caseyear,st_case,statename,drdistract,drdistractname
321862,2016,470473,Tennessee,99,Unknown if Distracted
219087,2014,470021,Tennessee,99,Unknown if Distracted
219066,2014,470004,Tennessee,99,Unknown if Distracted
219067,2014,470005,Tennessee,99,Unknown if Distracted
388539,2018,50370,Arkansas,99,Reported as Unknown if Distracted


In [5]:
#trim off data
df=pd.read_csv('data/VIOLATION.csv')
#fill NA value
df['violation'].fillna(df['mviolatn'],inplace=True)
df['violationname'].fillna(df['mviolatnname'],inplace=True)
df=df[['caseyear','st_case','statename','violation','violationname']]
#remove duplicate values
df=df.sort_values('violation',ascending=False)
df=df.drop_duplicates(subset=['caseyear','st_case'])
df.to_csv('data/VIOLATION.csv',index=False)
df.head()

c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,caseyear,st_case,statename,violation,violationname
154947,2013,90094,Connecticut,99.0,Unknown Violation(s)
99291,2012,40630,Arizona,99.0,Unknown Violation(s)
170215,2013,270296,Minnesota,99.0,Unknown Violation(s)
442113,2018,320277,Nevada,99.0,Unknown Violation(s)
499905,2019,360050,New York,99.0,Unknown Violation(s)


In [7]:
#trim off data
df=pd.read_csv('data/PERSON.csv')
df=df[['caseyear','st_case','age','sexname','per_typ','per_typname','inj_sev','inj_sevname','seat_pos','seat_posname']]
#only take people inside in-transport vehicle in crash
df=df.loc[(df['per_typname']=='Unknown Occupant Type in a Motor Vehicle In- Transport') |
       (df['per_typname']=='Driver of a Motor Vehicle In-Transport') |
       (df['per_typname']=='Passenger of a Motor Vehicle In-Transport'),:]
#replace the term for readability
df.replace({'Driver of a Motor Vehicle In-Transport':'Driver',
            'Passenger of a Motor Vehicle In-Transport':'Passenger',
            'Unknown Occupant Type in a Motor Vehicle In- Transport':'Unknown'},inplace=True)
df.to_csv('data/PERSON.csv')

c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (8,17,19,84,86,88,90,92,94,113,115,117,118,124,128,130) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
